# Power calculations

This model serves as a tool to check the model output of the master thesis work of Loes Segers.

### Loading packages

In [1]:
#Packages 

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math 

### Fixed input values

In [2]:
#Fixed input values

viscosity = 1*10**(-6) #kinematic viscosity
rho = 1000
g = 9.81
x = 2 #number of screws
eta_0 = 0.6     #open water eff
eta_r = 1.00     #relative rotative eff
eta_t = 0.98     #transmission eff
eta_g = 0.96     #gearing eff

In [3]:
#Fixed input values, ship related

L = 82.5
B = 8.2
T = 2.7

C_B = 0.85      #block coefficient 

P_installed = 640 #kW  

c_stern = 0

one_k2 = 2.5 #appendage resistance factor

In [4]:
#Ship related values, calculated from fixed input values

C_M = 1.006 - 0.0056 * C_B **(-3.56)
C_wp = (1+2 * C_B)/3
C_p = C_B / C_M

delta = C_B * L * B * T #water displacement

lcb = -13.5 + 19.4 * C_p
L_R = L * (1 - C_p + (0.06 * C_p * lcb)/(4 * C_p -1))

A_T = 0.2 * B * T

S_T = L*(2*T + B) * np.sqrt(C_M) * (0.453 + 0.4425 * C_B - 0.2862* C_M - 0.003467 * (B/T) + 0.3696 * C_wp) #+ 2.38 * (A_BT / C_B)


S_APP = 0.05 * S_T

S_B = L * B

D_s = 0.7 * T

### Fixed velocity and water depth

In [5]:
#Variables
V_0 = 4.39
h = 15

#V_0 = np.linspace(0, 5.0, 101)
#h = np.linspace(3.0, 15.0, 101)

## Resistance calculations

Method based on the division of Holtrop & Mennen, with some adjustments made in order to incorporate the shallow water effects

### A) Karpov velocity correction for limited water depths

In [6]:
############ Additional wave making resistance - Karpov #############
R_e = V_0 * L / viscosity

def karpov(h, F_n):
    
    #The Froude number used in the Karpov method is the depth related froude number F_nh
    
    #The different alpha** curves are determined with a sixth power polynomial approximation in Excel
    #A distinction is made between different ranges of Froude numbers, because this resulted in a better approximation of the curve
    
    if F_n <= 0.4:
        
        if 0 <= h/T < 1.75:
            alpha_xx = (-4*10**(-12)) * F_n**3 - 0.2143 * F_n**2 -0.0643 * F_n + 0.9997
        if 1.75 <= h/T < 2.25:
            alpha_xx = -0.8333 * F_n**3 + 0.25 * F_n**2 - 0.0167 * F_n + 1
        if 2.25 <= h/T < 2.75:
            alpha_xx = -1.25 * F_n**4 + 0.5833 * F_n**3 - 0.0375 * F_n**2 - 0.0108 * F_n + 1
        if h/T >= 2.75:
            alpha_xx = 1
    
    if F_n > 0.4:
        if 0 <= h/T < 1.75:
            alpha_xx = -0.9274 * F_n**6 + 9.5953 * F_n**5 - 37.197 * F_n**4 + 69.666 * F_n**3 - 65.391 * F_n**2 + 28.025 * F_n - 3.4143
        if 1.75 <= h/T < 2.25:
            alpha_xx = 2.2152 * F_n**6 - 11.852 * F_n**5 + 21.499 * F_n**4 - 12.174 * F_n**3 - 4.7873 * F_n**2 + 5.8662 * F_n - 0.2652
        if 2.25 <= h/T < 2.75:
            alpha_xx = 1.2205 * F_n**6 - 5.4999 * F_n**5 + 5.7966 * F_n**4 + 6.6491 * F_n**3 - 16.123 * F_n**2 + 9.2016 * F_n - 0.6342
        if 2.75 <= h/T < 3.25:
            alpha_xx = -0.4085 * F_n**6 + 4.534 * F_n**5 - 18.443 * F_n**4 + 35.744 * F_n**3 - 34.381 * F_n**2 + 15.042 * F_n - 1.3807
        if 3.25 <= h/T < 3.75:
            alpha_xx = 0.4078 * F_n **6 - 0.919 * F_n**5 - 3.8292 * F_n**4 + 15.738 * F_n**3 - 19.766 * F_n**2 + 9.7466 * F_n - 0.6409
        if 3.75 <= h/T < 4.5:
            alpha_xx = 0.3067 * F_n**6 - 0.3404 * F_n**5 - 5.0511 * F_n**4 + 16.892 * F_n**3 - 20.265 * F_n**2 + 9.9002 * F_n - 0.6712
        if 4.5 <= h/T < 5.5:
            alpha_xx = 0.3212 * F_n**6 - 0.3559 * F_n**5 - 5.1056 * F_n**4 + 16.926 * F_n**3 - 20.253 * F_n**2 + 10.013 * F_n - 0.7196
        if 5.5 <= h/T < 6.5:
            alpha_xx = 0.9252 * F_n**6 - 4.2574 * F_n**5 + 5.0363 * F_n **4 + 3.3282 * F_n**3 - 10.367 * F_n**2 + 6.3993 * F_n - 0.2074
        if 6.5 <= h/T < 7.5:
            alpha_xx = 0.8442 * F_n**6 - 4.0261 * F_n**5 + 5.313 * F_n **4 + 1.6442 * F_n**3 - 8.1848 * F_n**2 + 5.3209 * F_n - 0.0267
        if 7.5 <= h/T < 8.5:
            alpha_xx = 0.1211 * F_n**6 + 0.628 * F_n**5 - 6.5106 * F_n**4 + 16.7 * F_n**3 - 18.267 * F_n**2 + 8.7077 * F_n - 0.4745
        
                
        if 8.5 <= h/T < 9.5:
            if F_n < 0.6:
                alpha_xx = 1
            if F_n >= 0.6:
                alpha_xx = -6.4069 * F_n**6 + 47.308 * F_n**5 - 141.93 * F_n**4 + 220.23 * F_n**3 - 185.05 * F_n**2 + 79.25 * F_n - 12.484
        if h/T >= 9.5: 
            if F_n < 0.6:
                alpha_xx = 1
            if F_n >= 0.6:
                alpha_xx = -6.0727 * F_n**6 + 44.97 * F_n**5 - 135.21 * F_n**4 + 210.13 * F_n**3 - 176.72 * F_n**2 + 75.728 * F_n - 11.893
    
    return alpha_xx


In [7]:
F_nh = V_0 / np.sqrt(g * h)

#The corrected velocty V_2, based on the Karpov method:

V_2 = V_0 / karpov(h, F_nh)

### B) Frictional resistance (including shallow water effect by method of Zeng)

In [8]:
#Section 2.2 Resistance

#Section 2.2.1 Frictional resistance

R_e = V_0 * L / viscosity
D = h - T #distance from bottom ship to the bottom of the fairway

Cf_0 = 0.075 / ((math.log10(R_e)-2)**2)
print('Cf_0 is', Cf_0)

Cf_proposed = (0.08169/((np.log10(R_e) - 1.717)**2)) * (1 + (0.003998/(math.log10(R_e) - 4.393)) * (D/L)**(-1.083))

a = 0.042612 * math.log10(R_e) + 0.56725

Cf_katsui = 0.0066577 / ((math.log10(R_e) - 4.3762)**a)

if h/T <= 4:
    V_B = 0.4277 * V_0 * np.exp((h / T)**(-0.07625))
else:
    V_B = V_0

C_f = Cf_0 + (Cf_proposed - Cf_katsui) * (S_B / S_T) * (V_B / V_0)**2
print('C_f is', C_f)

R_f = (C_f * 0.5 * rho * (V_0**2) * S_T) / 1000 #kN
print('Total frictional resistance is', R_f)

print('The velocity V_0 is', V_0)
print('The increased velocity V_B is', V_B)

#R_f3 = 53 * (math.log10(V_0 * L) + 4)**(-2) * (L * B + 2 * L * T) * V_0**2

#print(R_f2 / 1000)
#print(R_f3 / 1000)
#print(S_T, S_2)

Cf_0 is 0.0017433990242849379
C_f is 0.0017460131648994659
Total frictional resistance is 16.31842971180384
The velocity V_0 is 4.39
The increased velocity V_B is 4.39


### C) Viscous resistance

In [9]:
#Section 2.2.2 Viscous resistance

c_14 = 1 + 0.0011 * c_stern
one_k1 = 0.93+0.487 * c_14 * ((B/L)**1.068) * ((T/L)**0.461) * ((L/L_R)**0.122) * (((L**3)/delta)**0.365) * ((1 - C_p)**(-0.604))

print(one_k1)

1.2130456604817983


### D) Appendage resistance 

In [10]:
#Section 2.2.3 Appendage resistance

R_APP = (0.5 * rho * (V_0**2) * S_APP * one_k2 * C_f) / 1000 #kN

print(R_APP)

2.03980371397548


### E) Wave resistance (dependend on V2)

In [11]:
def wave_resistance(V_2):

#This method is based on literature of Sarris (Naval Ship Propulsion and Electric Power Systems Selection for
#Optimal Fuel Consumption) (2003) applying Holtrop and Mennen (pag 26)

    F_n = V_2 / np.sqrt(g*L) #Froude number


    #parameter c_7 is determined by the B/L ratio
    if B/L < 0.11:
        c_7 = 0.229577 * (B/L)**0.33333
    if B/L > 0.25:
        c_7 = 0.5 - 0.0625 * (L/B)
    else: 
        c_7 = B/L

    #half angle of entrance in degrees
    i_E = 1 + 89 * np.exp(-((L/B)**0.80856) * ((1 - C_wp)**0.30484) * ((1 - C_p - 0.0225 * lcb)**0.6367) * ((L_R / B)**0.34574) * ((100 * delta / (L**3))**0.16302))

    c_1 = 2223105 * (c_7**3.78613) * ((T/B)**1.07961) * (90 - i_E)**(-1.37165) 
    c_2 = 1 #accounts for the effect of the bulbous bow, which is not present at inland ships
    c_5 = 1 - (0.8 * A_T) / (B * T * C_M) #influence of the transom stern on the wave resistance

    #parameter c_15 depoends on the ratio L^3 / delta
    if (L**3) / delta < 512:
        c_15 = -1.69385
    if (L**3) / delta > 1727:
        c_15 = 0
    else:
        c_15 = -1.69385 + (L / (delta**(1/3)) - 8) / 2.36

    #parameter c_16 depends on C_p
    if C_p < 0.8:
        c_16 = 8.07981 * C_p - 13.8673 * (C_p**2) + 6.984388 * (C_p**3)
    else:
        c_16 = 1.73014 - 0.7067

    m_1 = 0.0140407 * (L / T) - 1.75254 * ((delta)**(1/3) / L) - 4.79323 * (B / L) - c_16

    m_4 = 0.4 * c_15 * np.exp(-0.034 * (F_n**(-3.29)))    

    if L/B < 12:
        lmbda = 1.446 * C_p - 0.03 * (L/B)
    else: 
        lmbda = 1.446 * C_p - 0.036

    #parameters needed for RW_2
    c_17 = 6919.3 * (C_M**(-1.3346)) * ((delta/ (L**3))**2.00977) * ((L/B - 2)**1.40692)
    m_3 = -7.2035 * ((B / L)**0.326869) * ((T/B)**0.605375)


    ######### When Fn < 0.4
    RW_1 = c_1 * c_2 * c_5 * delta * rho * g * np.exp(m_1 * (F_n**(-0.9)) + m_4 * np.cos(lmbda * (F_n**(-2)))) 

    ######## When Fn > 0.5
    RW_2 = c_17 * c_2 * c_5 * delta * rho * g * np.exp(m_3 * (F_n**(-0.9)) + m_4 * np.cos(lmbda * (F_n**(-2))))
    
    if F_n < 0.4:
        R_W = RW_1 / 1000 #kN
    if F_n > 0.55:
        R_W = RW_2 / 1000 #kN
    else:
        R_W = (RW_1 + ((10 * F_n - 4) * (RW_2 - RW_1)) / 1.5) / 1000 #kN
        
    return R_W

In [12]:
R_W = wave_resistance(V_2)

### F) Residual resistance terms (dependend on V2)

In [13]:
#Resistance due to immersed transom
def resistance_transom(V_2):
    F_nt = V_2 / np.sqrt(2 * g * A_T / (B + B * C_wp))

    c_6 = 0.2 * (1 - 0.2 * F_nt)

    R_TR = (0.5 * rho * (V_2**2) * A_T * c_6) / 1000 #kN
    
    return R_TR

#Model-ship correlation resistance
def resistance_model_correlation(V_2):
    if T / L < 0.04:
        c_4 = T / L
    else:
        c_4 = 0.04
    c_2 = 1
    
    C_A = 0.006 * (L + 100)**(-0.16) - 0.00205 + 0.003 * np.sqrt(L / 7.5) * (C_B**4) * c_2 * (0.04 - c_4)

    #C_A = (1.8 + 260/L) * 0.0001 #Holtrop 1977

    ####### Holtrop and Mennen in the document of Sarris, 2003 #######
    R_A = (0.5 * rho * (V_2**2) * S_T * C_A) / 1000 #kW
    
    return R_A

In [14]:
R_TR = resistance_transom(V_2)
R_A = resistance_model_correlation(V_2)

### Total resistance

In [15]:
#Section 2.2 Resistance

#Total resistance

R_tot = R_f * one_k1 + R_APP + R_W + R_TR + R_A


print('The frictional resistance, including the viscous effect is', R_f * one_k1)
print('The appendage resistance is', R_APP)
print('The wave resistance is', R_W)
print('The resistance of the immersed transom is', R_TR)
print('The model correlation resistance term is', R_A)
print('The total resistance is', R_tot)

The frictional resistance, including the viscous effect is 19.795000347780892
The appendage resistance is 2.03980371397548
The wave resistance is 15.574797922020595
The resistance of the immersed transom is 5.3607478677319556
The model correlation resistance term is 5.570502077595547
The total resistance is 48.340851929104474


In [16]:
#Section 2.1 Total required power

#2.1.1 Required power for systems on board
P_hotel = 0.05 * P_installed

#2.1.2 Required power for propulsion

#Effective Horse Power (EHP)
P_EHP = V_B * R_tot

dw = 0 

#dw = np.zeros(101)
#counter = 0 

#Calculation hull efficiency
#for i in F_n:
    #if i < 0.2:
        #dw[counter] = 0
    #else:
        #dw[counter] = 0.1
    
    #counter += 1
    
w = 0.11 * (0.16 / x) * C_B * np.sqrt((delta**(1/3)) / D_s) - dw

if x == 1:
    t = 0.6 * w * (1 + 0.67 * w)
else:
    t = 0.8 * w * (1 + 0.25 * w)
    
eta_h = (1 - t) / (1 - w)

#Delivered Horse Power (DHP)

P_DHP = P_EHP / (eta_0 * eta_r * eta_h)

#Brake Horse Power (BHP)
P_BHP = P_DHP / (eta_t * eta_g)

P_tot = P_hotel + P_BHP

print('The total power required is', P_tot, 'kW')

P_partial = P_tot / P_installed

print('The partial load is', P_partial)

The total power required is 406.56317730802033 kW
The partial load is 0.6352549645437817


## Required energy during delta t

In [17]:
#Testcase
delta_s = 3886.991675
delta_t = (delta_s / V_0) / 3600    #in h

#delta_t = (1195.9037) / 3600

delta_E = P_tot * delta_t
print(delta_E)

99.99415879257303


In [18]:
print(V_0)

4.39


## Emission calculations

In the following code, functions are defined to calculate the emission factor.

The emission factor depends on the age of the engine. The age of the engine is determined according to a weibull function. Depending on the weight class of the vessel (L1, L2, L3), the vessel has a different weibull function. With a random generator the age of the vessel is drawn. (see function: calculate_engine_age) With the age of the engine, the construction  year of the engine is calculated (c_year). 

For testing purposes, a cell is included were the construction year of the engine is hardcoded. 

After the construction year has been determined, the corresponding general emission factor can be determined (for CO2, NOX, PM10) - (see function: general_emissionfactor).

The general emission factor has to be applied by a correction factor, which accounts for the partial engine load. If the partial engine load is low, the correction factors are higher (so the engine works less efficient). 

In [19]:
#Age of vessel of weight class L1, L2 or L3 

def calculate_engine_age(L_w=1):
    #Determining which shape and scale factor to use, based on the weight class L_w = L1, L2 or L3
    if L_w == 1:     #Weight class L1
        k = 1.3
        lmb = 20.5
    if L_w == 2:     #Weight class L2
        k = 1.12
        lmb = 18.5
    if L_w == 3:     #Weight class L3
        k = 1.26
        lmb = 18.6
            
    #The age of the engine
    age = int(np.random.weibull(k)*lmb)
        
    #Current year (TO DO: fix hardcoded year)
    year = 2020
        
    #Construction year of the engine
    c_year = year - age   
    return c_year

    
print(calculate_engine_age())

1989


In [20]:
c_year = 2000 #hardcoded, so 'ignoring' the cell above

In [21]:
def general_emissionfactor(c_year, L_w):
    if c_year < 1974:
        EM_CO2 = 756
        EM_PM10 = 0.6
        EM_NOX = 10.8
    if 1975 <= c_year <= 1979:
        EM_CO2 = 730
        EM_PM10 = 0.6
        EM_NOX = 10.6
    if 1980 <= c_year <= 1984:
        EM_CO2 = 714
        EM_PM10 = 0.6
        EM_NOX = 10.4
    if 1985 <= c_year <= 1989:
        EM_CO2 = 698
        EM_PM10 = 0.5
        EM_NOX = 10.1
    if 1990 <= c_year <= 1994:
        EM_CO2 = 698
        EM_PM10 = 0.4
        EM_NOX = 10.1
    if 1995 <= c_year <= 2002:
        EM_CO2 = 650
        EM_PM10 = 0.3
        EM_NOX = 9.4
    if 2003 <= c_year <= 2007:
        EM_CO2 = 635
        EM_PM10 = 0.3
        EM_NOX = 9.2
    if 2008 <= c_year <= 2019:
        EM_CO2 = 635
        EM_PM10 = 0.2
        EM_NOX = 7
    if c_year > 2019:
        if L == 1:
            EM_CO2 = 650
            EM_PM10 = 0.1
            EM_NOX = 2.9
        else:
            EM_CO2 = 603
            EM_PM10 = 0.015
            EM_NOX = 2.4
    return EM_CO2, EM_PM10, EM_NOX

general_emf = general_emissionfactor(c_year, 1)

In [22]:
#Correctionfactor function, which reads the right values from the correction factor table, based on the partial engine load
#Partial engine load = P_partial
#And based on the construction year of the engine (c_year), which has influence on the correction factors of NOX

def correctionfactor(c_year, P_partial, L_w=1):
    
    corf = pd.read_excel (r'correctionfactors.xlsx')
    
    for i in range(20):
    #If the partial engine load is smaller or equal to 5%, the correction factors corresponding to P_partial = 5% are assigned.
        if P_partial <= corf.iloc[0, 0]:
            corf_CO2 = corf.iloc[0, 5]
            corf_PM10 = corf.iloc[0, 6]
            
            #The NOX correction factors are dependend on the construction year of the engine and the weight class
            if c_year < 2008:
                corf_NOX = corf.iloc[0, 1] #<= CCR-1 class
            if 2008 <= c_year <= 2019:
                corf_NOX = corf.iloc[0, 2] #CCR-2 / Stage IIIa
            if c_year > 2019:
                if L == 1:
                    corf_NOX = corf.iloc[0, 3] #Stage V: IWP/IWA-v/c-3 class (vessels with P <300 kW: assumed to be weight class L1)
                else:
                    corf_NOX = corf.iloc[0, 4]  #Stage V:IWP/IWA-v/c-4 class (vessels with P >300 kw: assumed to be weight class L2-L3)
    
        #If the partial engine load is greater than 5%:
        #It is determined inbetween which two percentages in the table the partial engine load lies
        #The correction factor is determined by means of linear interpolation  
        
        elif corf.iloc[i, 0] < P_partial <= corf.iloc[i + 1, 0]:
            corf_CO2 = ((P_partial - corf.iloc[i, 0]) * (corf.iloc[i+1, 5] - corf.iloc[i, 5])) / (corf.iloc[i+1, 0] - corf.iloc[i, 0]) + corf.iloc[i, 5]
            corf_PM10 = ((P_partial - corf.iloc[i, 0]) * (corf.iloc[i+1, 6] - corf.iloc[i, 6])) / (corf.iloc[i+1, 0] - corf.iloc[i, 0]) + corf.iloc[i, 6]
            
            if c_year < 2008:
                corf_NOX = ((P_partial - corf.iloc[i, 0]) * (corf.iloc[i+1, 1] - corf.iloc[i, 1])) / (corf.iloc[i+1, 0] - corf.iloc[i, 0]) + corf.iloc[i, 1]
            if 2008 <= c_year <= 2019:
                corf_NOX = ((P_partial - corf.iloc[i, 0]) * (corf.iloc[i+1, 2] - corf.iloc[i, 2])) / (corf.iloc[i+1, 0] - corf.iloc[i, 0]) + corf.iloc[i, 2]
            if c_year > 2019:
                if L_w == 1:
                    corf_NOX = ((P_partial - corf.iloc[i, 0]) * (corf.iloc[i+1, 3] - corf.iloc[i, 3])) / (corf.iloc[i+1, 0] - corf.iloc[i, 0]) + corf.iloc[i, 3]
                else:
                    corf_NOX = ((P_partial - corf.iloc[i, 0]) * (corf.iloc[i+1, 4] - corf.iloc[i, 4])) / (corf.iloc[i+1, 0] - corf.iloc[i, 0]) + corf.iloc[i, 4]
    return corf_CO2, corf_PM10, corf_NOX

corf = correctionfactor(c_year, P_partial, 1)

In [23]:
#The total emission factor is calculated by multiplying the general emission factor (EM_CO2 / EM_PM10 / EM_NOX)
# By the correction factor (corf_CO2 / corf_PM10 / corf_NOX)

Emf_CO2 = general_emf[0] * corf[0]
Emf_PM10 = general_emf[1] * corf[1]
Emf_NOX = general_emf[2] * corf[2]

print('The general emission factor of CO2 is', general_emf[0])
print('The general emission factor of PM10 is', general_emf[1])
print('The general emission factor of NOx is', general_emf[2])
print()
print('The correction factor of CO2 is', corf[0])
print('The correction factor of PM10 is', corf[1])
print('The correction factor of NOx is', corf[2])
print()
print('The total emission factor of CO2 is', Emf_CO2)
print('The total emission factor of PM10 is', Emf_PM10)
print('The total emission factor of NOX is', Emf_NOX)

The general emission factor of CO2 is 650
The general emission factor of PM10 is 0.3
The general emission factor of NOx is 9.4

The correction factor of CO2 is 1.0129490070912437
The correction factor of PM10 is 0.9929490070912437
The correction factor of NOx is 0.99

The total emission factor of CO2 is 658.4168546093084
The total emission factor of PM10 is 0.2978847021273731
The total emission factor of NOX is 9.306000000000001


## Total emissions during delta t

In [24]:
#We already calculated the delta_E (the energy needed during delta t)
#We can calculate the emissions by multiplying the delta_E [kWh] by the total emission factor [g / kWh]

delta_CO2 = delta_E * Emf_CO2
delta_PM10 = delta_E * Emf_PM10
delta_NOX = delta_E * Emf_NOX

print('The CO2 emission during delta t are', delta_CO2, 'g')
print('The PM10 emission during delta t are', delta_PM10, 'g')
print('The NOX emission during delta t are', delta_NOX, 'g')

The CO2 emission during delta t are 65837.83951150965 g
The PM10 emission during delta t are 29.786730206402865 g
The NOX emission during delta t are 930.5456417236848 g


In [25]:
#Total emission Zuid-Beveland route (+/- 70 km) (with 1 m/s, it takes a ship 70076 s)
tot_E = P_tot * (70076/ V_0 / 3600)
tot_CO2 = P_tot * (70076/ V_0 / 3600) * Emf_CO2
tot_PM10 = P_tot * (70076/ V_0 / 3600) * Emf_PM10
tot_NOX = P_tot * (70076/ V_0 / 3600) * Emf_NOX

print('P_tot is', P_tot)
print('P_partial is', P_partial)
print('tot_E is', tot_E)
print('tot_CO2 is', tot_CO2)
print('tot_PM10 is', tot_PM10)
print('tot_NOX is', tot_NOX)

P_tot is 406.56317730802033
P_partial is 0.6352549645437817
tot_E is 1802.7284999390556
tot_CO2 is 1186946.8286444298
tot_PM10 is 537.0052422208718
tot_NOX is 16776.191420432853


## Plots